# Files and paths

There are multiple ways that you can represent file system paths. Dealing with file paths correctly involves a bit of planning, especially if you intend for your code to be cross platform.

Thankfully since Python 3.4 the `pathlib` module has made this substantially easier to do. (If you have to support Python 2 there is also backported library called `pathlib2` which will provide these features.)

## Difficulties representing paths as strings

If you store paths in strings, a number of things can go wrong.

### Backslashes

Several problems occur because backslashes in strings introduce escape sequences.

As one example, this is a SyntaxError (in Python 3.x):

In [1]:
path = 'C:\Users\d123456'

Why? Python expects `\U` to be followed by a sequence of numbers to represent an arbitrary unicode character.

In this next example, `\n` is a newline and `\t` is a tab character:

In [2]:
path = 'C:\Documents and settings\norman\to_do_list.txt'
print(path)

C:\Documents and settings
orman	o_do_list.txt


### Using "raw" strings

Use an `r` prefix to designate a string as a "raw string" -- telling Python not to interpret backslashes as having a special meaning. This fixes the problem of escaping:

In [1]:
path = r'C:\Users\d12345'
print(path)

C:\Users\d12345


In [3]:
path = r'C:\Documents and settings\norman\to_do_list.txt'
print(path)

C:\Documents and settings\norman\to_do_list.txt


Beware that adding a trailing backslash character to the end of a string still requires special handling:

In [4]:
path = r'C:\Users\norman' + '\\'
print(path)

C:\Users\norman\


Raw strings are useful, especially for entering Windows paths and regular expressions. However, there are still multiple issues.

### Path differences across platforms

Differences in the path separator across platforms can also cause issues. For example:

In [37]:
auto_path = r"Data\Auto.csv"

# Works on Windows but fails on Mac / Linux:
import pandas as pd
pd.read_csv(auto_path)

FileNotFoundError: [Errno 2] No such file or directory: 'Data\\Auto.csv'

## Path operations with `pathlib`

Using `pathlib` is now the recommended way of dealing with paths in Python. It solves a number of the issues above by providing a dedicated `Path` data type distinct from strings.

In [30]:
import pathlib

data_path = pathlib.Path(".") / "Data"
data_path

PosixPath('Data')

`pathlib` contains a number of convenience functions for manipulating paths:

In [45]:
auto_path = data_path / 'Auto.csv'
csv = auto_path.read_text(encoding='utf-8')

In [46]:
# Convert to absolute path
data_abs = data_path.absolute()
data_abs

PosixPath('/home/janis/PythonCharmers/NetworkEngineers/Data')

In [50]:
# Find extensions
config_file = pathlib.Path("config.ini")
config_file.suffix

'.ini'

In [51]:
config_file.stem

'config'

It may be useful to get the current user's home directory -- this is usually where their files are kept. This can be done through the `.expanduser` method:

In [4]:
data_path = Path('~/Data').expanduser()
data_path

PosixPath('/Users/schofield/Data')

### Wildcard expansion (globbing)

``Path`` objects allow handy wildcard expansion ("globbing") for files and directories. Here is an example:

In [2]:
files = data_path.glob('*.csv')
filelist = list(files)
filelist[:5]

['./Data/states_only.csv',
 './Data/glass.csv',
 './Data/abalone.csv',
 './Data/ocean.csv',
 './Data/coil_2000.csv']

This works recursively too:

In [3]:
morefiles = data_path.glob('**/*.csv', recursive=True)
filenames[::6]

./Data/Rainfall/10011.csv
./Data/Rainfall/10010.csv
./Data/Rainfall/10012.csv
./Data/Rainfall/10013.csv
./Data/linear_bucket/test_data.csv
./Data/BayesianDataAnalysis/tulips.csv


### Exercise

Using `glob`, print out any filename that starts with an "a".

**Hint:** You'll need to use ``string.split()`` to get just the filename, along with negative indexing.

### Converting `Path` objects to strings

All standard library functions in recent Python versions (3.6 onwards) that accept file paths should operate with path-like objects.

If you do encounter an older or buggy 3rd-party library that requires a string to represent a file path, use the `os.fspath()` function to convert your path to a string. See PEP 519 for details.

## Reading text files

Text files are just binary files whose contents are valid "text" in some encoding. The most common text encodings for files are UTF-8 and ASCII. Other less common ones include UTF-16, Latin-1, and CJK.

The easiest way to read a text file's contents in Python is by calling the `read_text()` method on the `Path` object:

In [ ]:
airfares_path = data_path / 'airfares.txt'
airfares = airfares_path.read_text(encoding='utf8')
print(airfares[:72])

ABE   ATL     613.84   3627.18   692  215.77    102.17  188.12     87.06

Best practice is to specify the encoding always, to help make your code explicit and portable across platforms with different default encodings.

## Reading binary files

Binary files are any file that isn't stored as text, including compiled programs, Word documents, and database files. We can read binary files in and print out the results as follows:

In [57]:
testdata_path = data_path / 'testdata.bin'
data = testdata_path.read_bytes()
print(data[:50])

b'SQLite format 3\x00\x04\x00\x01\x01\x00@  \x00\x00\x00.\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x04\x00\x00'


The result of calling `.read_bytes()` on a `Path` is a `bytes` object. What is printed includes the data in the file and information about the structure. For example, the first few characters tell us it is a SQLite database, specifically using format 3. We can use this information to look-up the structure.

We may also want to just retrieve the strings from this file. We can do that by looking for groups of printable characters.

To explore the strings in this binary file further, we need to convert our bytes into a string, which we can do like this:

In [59]:
string_data = data.decode('ascii', errors='ignore')
string_data[:50]

'SQLite format 3\x00\x04\x00\x01\x01\x00@  \x00\x00\x00.\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x04\x00\x00'

In [61]:
from string import printable

current_word = ""
words = []
for letter in string_data:
    if letter in printable:
        current_word += letter
    else:
        if len(current_word) > 3:
            words.append(current_word)
            current_word = ""

if len(current_word) > 3:
    words.append(current_word)

words

['SQLite format 3',
 '@  .',
 '.-\rZ',
 'sZT%',
 'sindexdocument_idsdocuments',
 'CREATE UNIQUE INDEX document_ids ON documents (did)@',
 'Otabledocumentsdocuments',
 'CREATE TABLE documents (\n\tdid INTEGER NOT NULL, \n\ttext TEXT, \n\tuploaded DATETIME, \n\tuid INTEGER, \n\tPRIMARY KEY (did), \n\tFOREIGN KEY(uid) REFERENCES users (uid)\n)\n',
 'stableusersusers',
 'CREATE TABLE users (\n\tuid INTEGER NOT NULL, \n\tusername TEXT, \n\tpassword TEXT, \n\tprofile TEXT, \n\tPRIMARY KEY (uid)\n)\r',
 '""""',
 'x$Auser3{"freqs": {}, "weight": 2}$',
 'Auser2{"freqs": {}, "weight": 2}$',
 'Auser1{"freqs": {}, "weight": 2}\r',
 'c.33A',
 'Document 1 by user32015-05-20 01:46:26.134518',
 '33ADocument 0 by user32015-05-20 01:46:26.083578',
 '33ADocument 1 by user22015-05-20 01:46:26.039238',
 '33ADocument 0 by user22015-05-20 01:46:25.989405',
 '23A\tDocument 1 by user12015-05-20 01:46:25.9264112',
 '3A\tDocument 0 by user12015-05-20 01:46:25.860113\n']

If we know more about the structure, we can load the binary file using a library that supports it. For instance, we now know this is a SQLite database, so we can load it as such.

String searching can also be effective when the file has been corrupted. In `Data/corrupted.bin` we have corrupted the same file with some random data.

### Exercise

Read the strings from the corrupted file and see what was lost.

## Compressed data (zip, gzip, bz2, ...)

Python's standard library has modules for in-memory compression and decompression of data in different formats: ``gzip``, ``zip`` (PKZip), ``lzma``, ``bzip2``, and Unix ``tar``.

Here is an example of transparently decompressing data from a `gzip`-compressed file:

In [6]:
alice_gz_path = data_path / 'alice_in_wonderland.txt.gz'

import gzip
alice_data = gzip.GzipFile(alice_gz_path).read()
alice_data[:55]

b'\xef\xbb\xbfThe Project Gutenberg EBook of Alice in Wonderland, '

Notice that the result of calling `.read()` on the `GzipFile` is a `bytes` object (always) with the uncompressed data. If this represents text in some encoding (like UTF-8), you can decode it to a `str` object as follows:

In [8]:
alice_text = alice_data.decode('utf-8')
type(alice_text)

'﻿The Project Gutenberg EBook of Alice in Wonderland, by'

In case you are wondering, `\ufeff` is the "byte-order mark" often present in text files on Windows.

The other modules `bz2`, `lzma`, etc. have similar interfaces to `gzip` for their respective compression formats.

### Exercise

Open the `Data/500-worst-passwords.txt.bz2` file and list the first 10.

In [11]:
# See solutions/badpasswords.py

### Zip files

Zip files work a little differently, in that they contain *collections* of files, not
just a single file. The `zipfile` module works with zips:

In [11]:
import zipfile

data_path.glob('*.zip')[:6]

['Data/housesalesprediction.zip',
 'Data/twitter.json.zip',
 'Data/LoanStats3a.csv.zip',
 'Data/spambase.zip',
 'Data/world-university-rankings.zip',
 'Data/USStatesData.zip']

In [13]:
states_path = data_path / 'USStatesData.zip'
z = zipfile.ZipFile(states_path)
z.filelist

[<ZipInfo filename='States.csv' compress_type=deflate external_attr=0x20 file_size=3797 compress_size=1791>,
 <ZipInfo filename='States.json' compress_type=deflate external_attr=0x20 file_size=10875 compress_size=2060>,
 <ZipInfo filename='States.xlsx' compress_type=deflate external_attr=0x20 file_size=15239 compress_size=11856>,
 <ZipInfo filename='States.xml' compress_type=deflate external_attr=0x20 file_size=9834 compress_size=2108>]

In [19]:
z.extractall("output_folder")  # Helper function to simply extract all files to a directory

file = z.getinfo("States.csv")
s = z.extract(file)
s  # Gives filename

'/root/home/robertlayton/pythoncharmers/DataAnalytics/modules/2070 Standard library tour/States.csv'

In [31]:
buffer = z.open(file)
buffer.read()[:50]  # Read files without needing to unzip to file

b'name,abbreviation,capital,most populous city,popul'

### Exercise

Write a function `read_csv_from_zip(zip_filename, name)` which reads and returns
the given file `name` from inside a zip file like `huge_us_stocks_data.zip` as a
DataFrame.

In [ ]:
# See solutions/read_csv_from_zip.py

## Low-level file operations

If you are not using the simple `.read_text()` or `.read_bytes()` methods of `pathlib` or another library like *Pandas* for reading files, you can operate on file objects manually. This is lower-level, but is useful sometimes, such as for processing enormous log files (e.g. terabytes) line-by-line.

Before reading a text file's contents the low-level way in Python, you open the file in *text* mode. One way is as follows:

In [ ]:
file = open(airfares_path, mode='rt')  # rt means "read as text"
# Code to read the file's contents goes here
...
file.close()

In place of the ellipsis `...` you add code to read some or all of the file. You can read an entire text file into a string in Python as follows:

In [ ]:
contents = file.read()

Or, if the file is large, you can read the file one line at a time by iterating over the file:

In [ ]:
for line in file:
    # Code to process the line goes here:
    ...

To open a file for reading in *binary* mode, use the `'rb'` flags for the `mode` keyword argument:

In [ ]:
file = open(airfares_path, mode='rb')  # rb means "read as binary / bytes"
# Code to read the file's contents goes here
...
file.close()

The result of calling `.read()` on a file opened in binary mode is a `bytes` object.

### Automatically closing files

Files in Python are "context managers"; they can be used with Python's special `with` keyword to close themselves automatically at the end of a block of code. Best practice is therefore to read files like this:

In [ ]:
with open(airfares_path, mode='rt') as file:
    contents = file.read()

Notice that there is no need for `file.close()`. The file will be closed automatically at the end of the `with` block.

The low-level way to read the binary file from earlier is:

In [58]:
testdata_path = data_path / 'testdata.bin'
with open(testdata_path, 'rb') as inf:   # rb means "read as bytes"
    data = inf.read()
print(data[:50])

b'SQLite format 3\x00\x04\x00\x01\x01\x00@  \x00\x00\x00.\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x04\x00\x00'


### Low-level operations on compressed files

First, observe the result of using Python's built-in `open` function to read the first 50 bytes of a `gzip`-compressed file:

In [6]:
alice_gz_path = data_path / 'alice_in_wonderland.txt.gz'
with open(alice_gz_path, mode='rb') as f:
    print(f.readline()[:50])   # first 50 bytes

b'\x1f\x8b\x08\x08[3]O\x00\x03alice_in_wonderland.txt\x00\xcd\xfd\xdbr\x1b\xd7\x96-\x8a\xbe+B\xff\x90\xd4\x0b'


The result is incomprehensible compressed binary data. However, there is a corresponding `open` function in the `gzip` module that decompresses the file transparently in memory:

In [7]:
import gzip
with gzip.open(alice_gz_path, mode='rt') as f:
    print(f.readline())   # first line

ALICE'S ADVENTURES IN WONDERLAND



CPUs and memory are fast; disks are slow. It is considerably faster to decompress a large file on the fly in memory than to decompress it into a file on disk only to re-read it from disk back into memory (unless you will read it many times).

## Cryptographic hashing (optional topic)

A cryptographic hash is a fingerprint of some binary data like a file's contents
that is (for practical purposes) unique. It can be used to verify the integrity
of a file or blob of data: if the hash of two files is the same, the probability
that the files are identical is very high, so one of them is extremely unlikely
to have been corrupted or tampered with.

Common cryptographic hash functions include MD5, SHA1, and SHA256.

This example shows how to compute the MD5 hash of a file's contents

In [37]:
import hashlib

portfolio_path = data_path / 'portfolio_flat.h5'
data = portfolio_path.read_bytes()

myhash = hashlib.md5(data)
myhash.hexdigest()

'4044e2839de128940993f91c6a8a778a'

In [41]:
# Linux:
# !md5sum Data/portfolio_flat.h5
# macOS:
# !md5 Data/portfolio_flat.h5
# Windows: not available

MD5 (Data/portfolio_flat.h5) = 4044e2839de128940993f91c6a8a778a


Computing a hash is a quick way to test if two byte-strings are equivalent, as hashes of very long byte-strings will still be short (commonly 128 to 512 bits).

In [42]:
mystring = "The cat in the hat"

print(hashlib.md5(mystring.encode()).hexdigest())

962c41d7eb82948eb42b756bf0e74bd8


### Challenge exercise

Combine your ability to search for files with cryptographic hashing to find duplicate files in the `Data` directory.

Hint: get the hashing part working first, and create a helper function that takes a filename as input and returns as hash. Then work with a dictionary to store the mapping between a hash and a list of corresponding filenames.

**Hint:** Check out the `groupby` function in the `toolz` package.